In [2]:
%%writefile sorting_algorithms.cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
#include <fstream>
#include <random>
#include <cmath>
#include <future>
#include <sstream>

// Threshold constants
const int INSERTION_SORT_THRESHOLD = 10;
const int SMALL_ARRAY_THRESHOLD = 1000;

// ---------------------------
// Insertion Sort (in-place)
// ---------------------------
void insertionSort(std::vector<double>& arr, int start, int end) {
    for (int i = start + 1; i <= end; i++) {
        double key = arr[i];
        int j = i - 1;
        while (j >= start && arr[j] > key) {
            arr[j + 1] = arr[j];
            j--;
        }
        arr[j + 1] = key;
    }
}

// ---------------------------
// Merge routine for merge sort
// ---------------------------
void merge(std::vector<double>& arr, int start, int mid, int end) {
    int n1 = mid - start + 1;
    int n2 = end - mid;
    std::vector<double> L(n1);
    std::vector<double> R(n2);

    for (int i = 0; i < n1; i++) {
        L[i] = arr[start + i];
    }
    for (int j = 0; j < n2; j++) {
        R[j] = arr[mid + 1 + j];
    }

    int i = 0, j = 0, k = start;
    while (i < n1 && j < n2) {
        if (L[i] <= R[j])
            arr[k++] = L[i++];
        else
            arr[k++] = R[j++];
    }
    while (i < n1) {
        arr[k++] = L[i++];
    }
    while (j < n2) {
        arr[k++] = R[j++];
    }
}

// ---------------------------
// Standard Merge Sort
// ---------------------------
void mergeSort(std::vector<double>& arr, int start, int end) {
    if (start < end) {
        int mid = start + (end - start) / 2;
        mergeSort(arr, start, mid);
        mergeSort(arr, mid + 1, end);
        merge(arr, start, mid, end);
    }
}

// ---------------------------
// Merge Sort with Coarsening (switches to insertion sort for small arrays)
// ---------------------------
void mergeSortCoarsened(std::vector<double>& arr, int start, int end) {
    if (end - start <= INSERTION_SORT_THRESHOLD) {
        insertionSort(arr, start, end);
        return;
    }
    int mid = start + (end - start) / 2;
    mergeSortCoarsened(arr, start, mid);
    mergeSortCoarsened(arr, mid + 1, end);
    merge(arr, start, mid, end);
}

// ---------------------------
// Parallel Merge Sort using std::async
// ---------------------------
void parallelMergeSort(std::vector<double>& arr, int start, int end) {
    const int PARALLEL_THRESHOLD = 10000; // Adjust as needed
    if (end - start < PARALLEL_THRESHOLD) {
        mergeSortCoarsened(arr, start, end);
        return;
    }
    int mid = start + (end - start) / 2;
    // Launch left half in parallel
    auto futureLeft = std::async(std::launch::async, parallelMergeSort, std::ref(arr), start, mid);
    // Process right half in current thread
    parallelMergeSort(arr, mid + 1, end);
    futureLeft.get();
    merge(arr, start, mid, end);
}

// ---------------------------
// Optimized BoxSort Implementation
// ---------------------------
void optimizedBoxSort(std::vector<double>& arr, int start, int end) {
    int n = end - start + 1;
    if (n <= INSERTION_SORT_THRESHOLD) {
        insertionSort(arr, start, end);
        return;
    }
    if (n <= SMALL_ARRAY_THRESHOLD) {
        mergeSortCoarsened(arr, start, end);
        return;
    }

    int numBoxes = std::max(1, (int)std::sqrt(n));
    int blockSize = n / numBoxes;
    std::vector<double> pivots(numBoxes);
    // Select pivots from each block (choose middle element of block)
    for (int i = 0; i < numBoxes; i++) {
        int index = start + i * blockSize + blockSize / 2;
        if (index > end) index = end;
        pivots[i] = arr[index];
    }
    std::sort(pivots.begin(), pivots.end());

    // Determine box membership for each element
    std::vector<int> boxSizes(numBoxes + 1, 0);
    std::vector<int> boxMembership(n, 0);
    for (int i = start; i <= end; i++) {
        double x = arr[i];
        int box = 0;
        if (pivots.empty() || x < pivots[0]) {
            box = 0;
        } else if (x >= pivots[numBoxes - 1]) {
            box = numBoxes;
        } else {
            int left = 0, right = numBoxes - 1;
            while (left <= right) {
                int mid = left + (right - left) / 2;
                if (mid < numBoxes - 1 && x >= pivots[mid] && x < pivots[mid + 1]) {
                    box = mid + 1;
                    break;
                }
                if (x < pivots[mid])
                    right = mid - 1;
                else
                    left = mid + 1;
            }
            if (left > right)
                box = left;
        }
        boxMembership[i - start] = box;
        boxSizes[box]++;
    }

    // Compute starting indices for each box
    std::vector<int> boxStartIndices(numBoxes + 1, 0);
    boxStartIndices[0] = start;
    for (int i = 1; i <= numBoxes; i++) {
        boxStartIndices[i] = boxStartIndices[i - 1] + boxSizes[i - 1];
    }

    // Rearrangement: distribute elements into boxes in a temporary vector
    std::vector<double> temp(n);
    std::vector<int> currentPositions = boxStartIndices; // copy start indices
    for (int i = start; i <= end; i++) {
        int box = boxMembership[i - start];
        int pos = currentPositions[box] - start;
        temp[pos] = arr[i];
        currentPositions[box]++;
    }
    // Copy back from temp to arr segment
    for (int i = 0; i < n; i++) {
        arr[start + i] = temp[i];
    }

    // Recursively sort each box
    for (int i = 0; i <= numBoxes; i++) {
        int boxStart = boxStartIndices[i];
        int boxEnd = (i < numBoxes) ? boxStartIndices[i + 1] - 1 : end;
        if (boxStart < boxEnd)
            optimizedBoxSort(arr, boxStart, boxEnd);
    }
}

// ---------------------------
// C++ Built-in Sort (std::sort)
// ---------------------------
void cppBuiltInSort(std::vector<double>& arr) {
    std::sort(arr.begin(), arr.end());
}

// ---------------------------
// Utility: Check if sorted
// ---------------------------
bool isSorted(const std::vector<double>& arr) {
    for (size_t i = 1; i < arr.size(); i++) {
        if (arr[i - 1] > arr[i])
            return false;
    }
    return true;
}

// ---------------------------
// Benchmarking and CSV Output in main()
// ---------------------------
int main() {
    int iterations = 10;  // Number of iterations per algorithm (iteration 1 = warmup)
    std::ofstream csvWriter("sorting_benchmark_results_cpp.csv");
    if (!csvWriter.is_open()) {
        std::cerr << "Error opening CSV file for writing.\n";
        return 1;
    }
    csvWriter << "Algorithm,Size,Iteration,Time(ms),IsWarmup\n";

    // We'll test sizes 10^3 up to 10^8 (adjust exponents as needed)
    for (int exp = 3; exp <= 8; exp++) {
        int n = (int)std::pow(10, exp);
        std::cout << "Testing array size n = " << n << "\n";

        // Create a reproducible random array.
        std::vector<double> original(n);
        std::mt19937 rng(42);
        std::uniform_real_distribution<double> dist(0.0, 1.0);
        for (int i = 0; i < n; i++) {
            original[i] = dist(rng);
        }

        // Define lambda to run a given sorting function and log timing.
        auto runSort = [&](const std::string &algName, void (*sortFunc)(std::vector<double>&)) {
            for (int iter = 1; iter <= iterations; iter++) {
                bool isWarmup = (iter == 1);
                std::vector<double> arr = original; // copy original array
                auto startTime = std::chrono::steady_clock::now();
                sortFunc(arr);
                auto endTime = std::chrono::steady_clock::now();
                auto runtime = std::chrono::duration_cast<std::chrono::milliseconds>(endTime - startTime).count();
                // Log CSV row
                csvWriter << algName << "," << n << "," << iter << "," << runtime << "," << (isWarmup ? "true" : "false") << "\n";
                // Optionally, verify sorted order:
                if (!isSorted(arr)) {
                    std::cerr << algName << " failed to sort array of size " << n << "\n";
                }
            }
            csvWriter.flush();
        };

        // Run each algorithm.
        runSort("mergeSort", [](std::vector<double>& arr) { mergeSort(arr, 0, arr.size() - 1); });
        runSort("mergeSortCoarsened", [](std::vector<double>& arr) { mergeSortCoarsened(arr, 0, arr.size() - 1); });
        runSort("parallelMergeSort", [](std::vector<double>& arr) { parallelMergeSort(arr, 0, arr.size() - 1); });
        runSort("cppBuiltInSort", cppBuiltInSort);
        runSort("optimizedBoxSort", [](std::vector<double>& arr) { optimizedBoxSort(arr, 0, arr.size() - 1); });
    }

    csvWriter.close();
    std::cout << "Benchmarking completed. Results saved to sorting_benchmark_results.csv\n";
    return 0;
}

Writing sorting_algorithms.cpp


In [3]:
!g++ -std=c++17 -O2 sorting_algorithms.cpp -o sorting_algorithms && ./sorting_algorithms

Testing array size n = 1000
Testing array size n = 10000
Testing array size n = 100000
Testing array size n = 1000000
Testing array size n = 10000000
Testing array size n = 100000000
Benchmarking completed. Results saved to sorting_benchmark_results.csv
